<a href="https://colab.research.google.com/github/antreev-brar/ModelZoo/blob/master/ResNet_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input, Conv2D, ReLU, BatchNormalization, AveragePooling2D, Flatten, Dense ,Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import os

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape[1:])
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

x_train = x_train.astype("float32") /255
x_test = x_test.astype("float32") /255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

input_shape = x_train.shape[1:]


(32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [0]:
def convblock(X ,filte, f , s):
  
  X = Conv2D(filters = filte ,
             kernel_size = (f,f) ,
             strides = (s,s) ,
             padding= "same",
             kernel_initializer='he_normal',
             kernel_regularizer=l2(1e-4) )(X)
  return X

def resblock(X_input ,flite , downscale):
  X_shortcut = X_input
  if downscale :
    X = convblock(X_input , flite ,3, 2 )
  else :
    X = convblock(X_input , flite ,3, 1 )

  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  
  X = convblock(X , flite ,3, 1 )
  X = BatchNormalization()(X)
  
  if downscale :
    X_shortcut =convblock(X_shortcut ,flite , 1 , 2 )
  print(X.shape)
  print(X_shortcut.shape)
  y = keras.layers.add([X, X_shortcut])
  y = Activation('relu')(y)
  return y

In [12]:
def lr_schedule(epoch):
  lr = 0.001 #standard rate for adam
  if epoch > 25 :
    lr = 0.0001
  if epoch >35 :
    lr = 0.00001
  if epoch >45 :
    lr = 0.000003
  if epoch >55 :
    lr = 0.000001
  print("learning rate =", lr)
  return lr
#---------------------------------------------------------
def ResNet(input_shape):
     inputs = Input(shape=input_shape)
     X_input = inputs

     #######here we go
     X = convblock(X_input , 32 ,3,1)
     X = BatchNormalization()(X)
     X = Activation('relu')(X)
     
     #first stage
     X = resblock( X , 32 ,downscale = False)
     X = resblock( X , 32 ,False)
     X = resblock( X , 32 ,False)
     X = resblock( X , 32 ,False)
 
     print(X.shape)
     #second stage
     X = resblock( X , 64 ,True)
     X = resblock( X , 64 ,False)
     X = resblock( X , 64 ,False)
     X = resblock( X , 64 ,False)
   
     print(X.shape)
     #third stage
     X = resblock( X , 128 ,True)
     X = resblock( X , 128 ,False)
     X = resblock( X , 128 ,False) 
     X = resblock( X , 128 ,False) 
 
     print(X.shape)

     X = AveragePooling2D(8)(X)
     print(X.shape)
  
     X = Flatten()(X)
     outputs = Dense(10, activation='softmax' , kernel_initializer='he_normal')(X)
     model = Model(inputs = inputs, outputs = outputs )

     return model
#--------------------------------------------------------


model = ResNet(input_shape=input_shape)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()




depth = 32
model_type = 'ResNet%d' % (depth)


# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)


# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint , lr_reducer, lr_scheduler]



datagen = ImageDataGenerator(
        zca_epsilon=1e-06,
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',
        horizontal_flip=True,
    )
datagen.fit(x_train)



model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                        validation_data=(x_test, y_test),
                        epochs=60, verbose=1, workers=4,
                        callbacks=callbacks)


scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 32, 32, 32)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 16, 16, 64)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 8, 8, 128)
(None, 1, 1, 128)
learning rate = 0.001
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_154 (Conv2D)             (None, 32, 

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


1563/1563 [==============================] - 120s 77ms/step - loss: 1.2313 - accuracy: 0.6702 - val_loss: 2.0733 - val_accuracy: 0.4748
Epoch 3/100
learning rate = 0.001
1563/1563 [==============================] - 120s 76ms/step - loss: 1.0498 - accuracy: 0.7338 - val_loss: 1.4940 - val_accuracy: 0.6322
Epoch 4/100
learning rate = 0.001
1563/1563 [==============================] - 120s 77ms/step - loss: 0.9597 - accuracy: 0.7643 - val_loss: 1.0952 - val_accuracy: 0.7255
Epoch 5/100
learning rate = 0.001
1563/1563 [==============================] - 119s 76ms/step - loss: 0.8927 - accuracy: 0.7871 - val_loss: 1.4874 - val_accuracy: 0.6522
Epoch 6/100
learning rate = 0.001
1563/1563 [==============================] - 120s 77ms/step - loss: 0.8521 - accuracy: 0.8019 - val_loss: 1.1621 - val_accuracy: 0.6998
Epoch 7/100
learning rate = 0.001
1563/1563 [==============================] - 119s 76ms/step - loss: 0.8169 - accuracy: 0.8141 - val_loss: 1.2512 - val_accuracy: 0.7031
Epoch 8/100
le

KeyboardInterrupt: ignored